# Input functionality testing notebook

In [11]:
%load_ext autoreload
%autoreload 2

Warning! Sigma.SEED = 5000
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# QuantumMASALA imports
from quantum_masala import config
from quantum_masala import pw_logger
from quantum_masala.constants import RYDBERG, ELECTRONVOLT

## EpsilonInp and SigmaInp

BerkeleyGW's input files, as per their specifications:
- http://manual.berkeleygw.org/3.0/epsilon-keywords/
- http://manual.berkeleygw.org/3.0/sigma-keywords/

In [13]:
from quantum_masala.gw.io_bgw import inp

# datadir = "./test/bgw/"
datadir = "./scripts/results/si_4_2ryd/si_4_gw/"

epsinp = inp.read_epsilon_inp(filename=f'{datadir}epsilon.inp')

# Epsilon.inp data
# Use __doc__ to print elements
print(epsinp.__doc__)
print(epsinp.options)
print()

# Sigma.inp data
sigmainp = inp.read_sigma_inp(filename=f'{datadir}sigma.inp')
print(sigmainp.__doc__)
print(sigmainp.options)

# Find how to specify docstring for each of these attributes...
# What is compulsory

EpsilonInp(epsilon_cutoff, number_bands, qpts, is_q0, options)
['use_wfn_hdf5', 'write_vcoul']

SigmaInp(band_index_min, band_index_max, number_bands, frequency_dependence, exact_static_ch, bare_coulomb_cutoff, screened_coulomb_cutoff, tol_degeneracy, kpts, options)
['use_wfn_hdf5', 'write_vcoul']


## WFN, RHO and Vxc.dat
Read output generated by pw2bgw.x

This includes:
- ``WFN.h5``: Mean field wavefunction and crystal data 
- ``RHO``: Density corresponding to wavefunctions from mean-field codes
- ``vxc.dat``: Exchange correlation energy values from mean-field codes

In [14]:
# !pip3 install --editable ../../QuantumMASALA


# wfn2py
from quantum_masala.gw.io_bgw.wfn2py import wfn2py

# RHO data
rho_data = inp.read_rho(f"{datadir}RHO")
print(rho_data.__doc__)

# Vxc data
vxc_data = inp.read_vxc(f'{datadir}vxc.dat')
print(vxc_data.__doc__)

# WFN data
wfndata = wfn2py(f'{datadir}WFN.h5')
print(wfndata.__doc__)

wfnqdata = wfn2py(f'{datadir}WFNq.h5')
print(wfnqdata.__doc__)

RHO(rho, gvecs)
VXC(vxc, kpts)
WfnData(crystal, grho, kpts, l_gk, l_wfn, symmetry)
WfnData(crystal, grho, kpts, l_gk, l_wfn, symmetry)


In [15]:
import numpy as np
bvec = np.array([-1,-1,-1])
# print(wfndata)
print(wfndata.crystal.recilat.bdot)
qsquared = bvec.T @ wfndata.crystal.recilat.bdot @ bvec
print(qsquared)
print(8*np.pi/qsquared)


[[ 1.13836268 -0.37945423 -0.37945423]
 [-0.37945423  1.13836268 -0.37945423]
 [-0.37945423 -0.37945423  1.13836268]]
1.1383626760195342
22.077973705707713


In [16]:
# print(vxc_data)

In [17]:
wfndata.grho.recilat.metric

array([[ 1.13836268, -0.37945423, -0.37945423],
       [-0.37945423,  1.13836268, -0.37945423],
       [-0.37945423, -0.37945423,  1.13836268]])

## QPoints

In [18]:
# QPoints

from quantum_masala.gw.core import QPoints

qpts = QPoints.from_cryst(wfndata.kpts.recilat, epsinp.is_q0, *epsinp.qpts)
print(qpts.q0vec)
print(qpts.index_q0)
print("head")
print(qpts.cryst.T[:5])
print("tail")
print(qpts.cryst.T[-5:])

q0vec init: [0.    0.    0.001]
[0.    0.    0.001]
0
head
[[0.    0.    0.001]
 [0.    0.    0.25 ]
 [0.    0.    0.5  ]
 [0.    0.    0.75 ]
 [0.    0.25  0.   ]]
tail
[[0.75 0.5  0.75]
 [0.75 0.75 0.  ]
 [0.75 0.75 0.25]
 [0.75 0.75 0.5 ]
 [0.75 0.75 0.75]]


## GSpaceQpt

In [19]:
# List of GSpaceQpt
# from quantum_masala.gw.core import GSpaceQpt
from quantum_masala.core import GSpace, GkSpace
gspc = wfndata.grho

print(wfndata.grho.grid_shape)
print(epsinp.epsilon_cutoff)

# print(qpts.cryst.T)

for qpt in qpts.cryst.T[:10]:
    # gspc_q = GSpaceQpt(wfndata.crystal.recilat, epsinp.epsilon_cutoff, wfndata.grho.grid_shape, qpt)
    gspc_q = GkSpace(gspc=wfndata.grho, k_cryst=qpt)
    # print(wfndata.)

    print(gspc_q.numgk, qpt)

(24, 24, 24)
2.0
537 [0.    0.    0.001]
564 [0.   0.   0.25]
568 [0.  0.  0.5]
564 [0.   0.   0.75]
564 [0.   0.25 0.  ]
560 [0.   0.25 0.25]
558 [0.   0.25 0.5 ]
554 [0.   0.25 0.75]
568 [0.  0.5 0. ]
558 [0.   0.5  0.25]


## Vcoul

In [20]:
from quantum_masala.gw.vcoul import Vcoul
import numpy as np

# WARNING: BGW's default settings have no avgcut
vcoul = Vcoul(gspace=wfndata.grho, qpts=qpts, bare_coulomb_cutoff=sigmainp.bare_coulomb_cutoff, avgcut=0.1)

Vcoul calculation for qpts:   0%|          | 0/64 [00:00<?, ?it/s]/home/agrim/Documents/qtmgw/QuantumMASALA/src/quantum_masala/gw/vcoul.py:761: RuntimeWarning: divide by zero encountered in double_scalars
  oneoverqsph = 8 * np.pi / shift_length
Vcoul calculation for qpts:   2%|▏         | 1/64 [00:03<03:53,  3.71s/it]

 0.00000000  0.00000000  0.00100000       0      0      0       3.26668818E+03
 0.00000000  0.00000000  0.00100000      -1     -1     -1       2.22003386E+01
 0.00000000  0.00000000  0.00100000      -1      0      0       2.21084154E+01
 0.00000000  0.00000000  0.00100000       0     -1      0       2.22120982E+01
 0.00000000  0.00000000  0.00100000       0      0     -1       2.22243816E+01
 0.00000000  0.00000000  0.00100000       0      0      1       2.21058782E+01
 0.00000000  0.00000000  0.00100000       0      1      0       2.21307837E+01
 0.00000000  0.00000000  0.00100000       1      0      0       2.22360143E+01
 0.00000000  0.00000000  0.00100000       1      1      1       2.21293639E+01
 0.00000000  0.00000000  0.00100000      -1     -1      0       1.65839529E+01
 0.00000000  0.00000000  0.00100000      -1      0     -1       1.66339164E+01
 0.00000000  0.00000000  0.00100000       0     -1     -1       1.67139975E+01
 0.00000000  0.00000000  0.00100000       0      1  

Vcoul calculation for qpts:   3%|▎         | 2/64 [00:07<03:50,  3.72s/it]

 0.00000000  0.00000000  0.25000000       0      0      0       3.77458243E+02
 0.00000000  0.00000000  0.25000000      -1     -1     -1       2.47786109E+01
 0.00000000  0.00000000  0.25000000      -1      0      0       1.79737559E+01
 0.00000000  0.00000000  0.25000000       0     -1      0       1.80241850E+01
 0.00000000  0.00000000  0.25000000       0      0     -1       3.94623317E+01
 0.00000000  0.00000000  0.25000000       0      0      1       1.41171291E+01
 0.00000000  0.00000000  0.25000000       0      1      0       2.47716075E+01
 0.00000000  0.00000000  0.25000000       1      0      0       2.47524331E+01
 0.00000000  0.00000000  0.25000000       1      1      1       1.79714097E+01
 0.00000000  0.00000000  0.25000000      -1     -1      0       1.27678184E+01
 0.00000000  0.00000000  0.25000000      -1      0     -1       2.08310161E+01
 0.00000000  0.00000000  0.25000000       0     -1     -1       2.09480558E+01
 0.00000000  0.00000000  0.25000000       0      1  

Vcoul calculation for qpts:   6%|▋         | 4/64 [00:07<01:58,  1.97s/it]


KeyboardInterrupt: 

In [ ]:
print(vcoul)
print(type(vcoul.l_gspace_q[0]))
# print(vcoul.l_gspace_q[0].cryst.T)

Vcoul:
            * gspace = <quantum_masala.core.gspc.gspc.GSpace object at 0x7fe72c13c740>
            * qpts = <quantum_masala.gw.core.QPoints object at 0x7fe6e13e6910>
            * bare_coulomb_cutoff = 2.0
            * avgcut = 0.1
            * l_gspace_q = <class 'list'> of length 64
            * vcoul = <class 'list'> of length 64
        
<class 'quantum_masala.core.gspc.gkspc.GkSpace'>


In [ ]:
from quantum_masala.gw.epsilon import Epsilon
epsmats = Epsilon.read_epsmat(None, f"{datadir}eps0mat.h5") \
        + Epsilon.read_epsmat(None, f"{datadir}epsmat.h5")
print(len(epsmats))
# epsmat = epsmat[0,0]


# i_q = 0
# import sympy
for i_q in range(4):
    print("before")
    print(np.real(epsmats[i_q][0,0,:2,:2]))
    fixed_epsinv = vcoul.calculate_fixedeps(epsinv=epsmats[i_q][0,0], i_q=i_q, random_sample=True)
    print("after")
    print(np.real(fixed_epsinv[:2,:2]))
    print()

10
before
[[ 7.69262298e-02 -1.13648386e-05]
 [-1.13572734e+01  7.44358846e-01]]
after
[[        nan -0.        ]
 [ 0.          0.74435885]]

before
[[0.13895176 0.0133853 ]
 [0.21416479 0.65128741]]
after
[[0.13895176 0.04663057]
 [0.17945862 0.65128741]]

before
[[0.29632739 0.04743467]
 [0.10672801 0.49634804]]
after
[[0.29632739 0.04743467]
 [0.10672801 0.49634804]]

before
[[0.13644357 0.00185265]
 [0.02732654 0.66456956]]
after
[[0.13644357 0.00655707]
 [0.02141189 0.66456956]]



/home/agrim/Documents/qtmgw/QuantumMASALA/src/quantum_masala/gw/vcoul.py:333: RuntimeWarning: invalid value encountered in cdouble_scalars
  return np.ones_like(arr) * wcoul0 / vcoul


In [ ]:
vcoul.write_vcoul()

#### Rough scripts

In [ ]:
# Read epsinv output from BGW to check by hand, to get rid of any stochastic disagreements
import numpy as np
def read_epsinv_from_sigma_out(dir="../scripts/results/si_4_gw/"):
    epsinv_data = []
    with open(file=f"{dir}sigma_out_seed.log") as f:
        lines = f.readlines()
        for i_line in range(225, 363):
            line = lines[i_line].strip()
            if not line or not line.startswith("("):
                continue
            eps_arr = line.split()
            L = len(eps_arr)
            epsinv_data.append(np.zeros(L, dtype = complex))
            for i, eps_ele in enumerate(eps_arr):
                eps_ele = eps_ele[1:-1].split(",")
                # print(eps_ele)
                epsinv_data[-1][i] = (complex(eps_ele[0])+1j*complex(eps_ele[1]))

            epsinv_data[-1] = epsinv_data[-1].reshape(int(L**0.5), -1)
    return epsinv_data

epsinv_data = read_epsinv_from_sigma_out()
print(epsinv_data)
np.save("./epsinvdata.npy",epsinv_data, allow_pickle=False)

FileNotFoundError: [Errno 2] No such file or directory: '../scripts/results/si_4_gw/sigma_out_seed.log'

In [ ]:
in_epsinv_data = np.load("./epsinvdata.npy")
# print(in_epsinv_data)
for ele in in_epsinv_data:
    print(in_epsinv_data.shape)

In [ ]:
# fact = 1D0/(dble(gr%nf-sig%nq0+1)*crys%celvol)
# coulfact = 8D0*PI_D/(dble(gr%nf-sig%nq0+1)*crys%celvol)

coulfact = 8*np.pi/(wfndata.crystal.reallat.cellvol*qpts.numq)
print("coulfact",coulfact)
sigma_factor = 17.307889/21599.51290058
print("sigma_factor:",sigma_factor )
# (coulfact/sigma_factor)**3/np.pi
fact = 1/(wfndata.crystal.reallat.cellvol*qpts.numq)
print("fact:", fact)
print("sigma_factor", sigma_factor)
print(sigma_factor/fact)
emf_factor = 5.790268/0.4255769
print(emf_factor)
print(coulfact*1.5**-1.5/sigma_factor)
# fact
# sigma_factor
print(fact*emf_factor/sigma_factor)

In [ ]:
1.5**1.5


In [ ]:
13.023507

sigma_sx = np.array([4.4713161e+01-0.00e+00j, 3.0392166e+01+0.00e+00j, 3.0398752e+01-0.00e+00j, 3.0400866e+01+0.00e+00j, 1.2236813e+01+0.00e+00j, 1.2242071e+01-0.00e+00j, 1.2234284e+01+0.00e+00j, 1.2530080e+01-0.00e+00j])
sigma_x = np.array([-60.267315+0.j, -43.703574+0.j, -43.703684+0.j, -43.703893+0.j, -20.116378+0.j, -20.117153+0.j, -20.11712 +0.j, -20.897789+0.j])

sigma_sx*13.023507/sigma_sx[0]
print(13.023507/4.4713161)
print(17.540362/60.267315)

print(sigma_factor)

print(1.5**-3)


In [ ]:
13.605691**-3

In [ ]:
print(sigma_sx*sigma_factor)
print(sigma_sx*1.5**-3)
print(sigma_x*1.5**-3)